In [7]:
# ---------------- #
# Common Libraries #
# ---------------- #
      
# Standard Imports
import os
import re
import nltk
import requests
import unicodedata
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords

## setting basic style parameters for matplotlib
plt.rc('figure', figsize=(13, 7))
plt.style.use('seaborn-darkgrid')

# ------------- #
# LOCAL IMPORTS #
# ------------- #

# importing sys
import sys
# adding 00_helper_files to the system path
sys.path.insert(0, '/Users/qmcbt/codeup-data-science/00_helper_files')
# env containing sensitive access credentials
import env
from env import user, password, host
from env import get_db_url
# Import Helper Modules
import QMCBT_00_quicktips as qt
import QMCBT_01_acquire as acq
import QMCBT_02_prepare as prep
import QMCBT_03_explore as exp
import QMCBT_04_visualize as viz
import QMCBT_05_model as mod
import QMCBT_wrangle as w

In [8]:
# Read-in data from compressed file
df = pd.read_pickle('michelin_df.pickle')

In [9]:
# Display DataFrame
df.T

,0,1,2,3,4,5,6,7,8,9,...,6770,6771,6772,6773,6774,6775,6776,6777,6778,6779
name,King's Joy,Xin Rong Ji (Xinyuan South Road),Taian Table,Ultraviolet by Paul Pairet,Quince,Atelier Crenn,Addison,Manresa,Benu,SingleThread,...,Mimo,A Casa do Porco,Più Iguatemi,Bistrot de Paris,AE! Cozinha,Szóstka,Fiorentina,Zazie,Butchery & Wine,alewino
address,"2 Wudaoying Hutong, Beijing, China Mainland","1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","101-102, Building No. 1, Garden Office, No.161...",'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"470 Pacific Ave., San Francisco, 94133, USA","3127 Fillmore St., San Francisco, 94123, USA","5200 Grand Del Mar Way, San Diego, 92130, USA","320 Village Ln., Los Gatos, 95030, USA","22 Hawthorne St., San Francisco, 94105, USA","131 North St., Healdsburg, 95448, USA",...,"Rua Caconde 118, São Paulo, 01425-010, Brazil","Rua Araujo 124, São Paulo, 01220-020, Brazil","Avenida Brigadeiro Faria Lima 2232, São Paulo,...","Rua Augusta 2542, São Paulo, 01412-100, Brazil","Rua Áurea 285, São Paulo, 04015-070, Brazil","Plac Powstańców Warszawy 9, Warsaw, 00 039, Po...","ul. Grodzka 63, Cracow, 31 044, Poland","ul. Józefa 34, Cracow, 32 056, Poland","ul. Żurawia 22, Warsaw, 00 515, Poland","ul. Mokotowska 48, Warsaw, 00 543, Poland"
location,"Beijing, China Mainland","Beijing, China Mainland","Shanghai, China Mainland","Shanghai, China Mainland","San Francisco, USA","San Francisco, USA","San Diego, USA","Los Gatos, USA","San Francisco, USA","Healdsburg, USA",...,"São Paulo, Brazil","São Paulo, Brazil","São Paulo, Brazil","São Paulo, Brazil","São Paulo, Brazil","Warsaw, Poland","Cracow, Poland","Cracow, Poland","Warsaw, Poland","Warsaw, Poland"
price,¥¥¥,¥¥¥¥,¥¥¥¥,¥¥¥¥,$$$$,$$$$,$$$$,$$$$,$$$$,$$$$,...,NaN,NaN,NaN,NaN,NaN,€€,€€,€,€€,€€
cuisine,Vegetarian,Taizhou,Innovative,Innovative,"Contemporary, Californian","Contemporary, French","Contemporary, Californian","Contemporary, Californian","Asian, Contemporary","Contemporary, Californian",...,Modern Cuisine,Brazilian,"Italian, Creative",French,"Creative, Market Cuisine","Polish, Modern Cuisine","Creative, Polish",French,"Meats and Grills, Traditional Cuisine","Polish, Traditional Cuisine"
longitude,116.410004,116.450148,121.474049,121.48509,-122.403261,-122.43586,-117.198891,-121.98071,-122.39906,-122.869723,...,-46.658201,-46.644742,-46.688315,-46.665519,-46.645587,21.012698,19.938179,19.946949,21.015495,21.022057
latitude,39.946681,39.94638,31.221807,31.240358,37.797505,37.79835,32.941297,37.22761,37.785376,38.61226,...,-23.574572,-23.544867,-23.577106,-23.563072,-23.586913,52.235803,50.055898,50.05124,52.228581,52.225201
phone_number,861084049191.0,861065015501.0,8617301605350.0,NaN,14157758500.0,14154400460.0,18583141900.0,14083544330.0,14156854860.0,17077234646.0,...,551130522517.0,551132582578.0,551131987649.0,551130631675.0,551134768521.0,48224700342.0,48124264608.0,48500410829.0,48225023118.0,48226283830.0
url,https://guide.michelin.com/en/beijing-municipa...,https://guide.michelin.com/en/beijing-municipa...,https://guide.michelin.com/en/shanghai-municip...,https://guide.michelin.com/en/shanghai-municip...,https://guide.michelin.com/en/california/san-f...,https://guide.michelin.com/en/california/san-f...,https://guide.michelin.com/en/california/us-sa...,https://guide.michelin.com/en/california/los-g...,https://guide.michelin.com/en/california/san-f...,https://guide.michelin.com/en/california/heald...,...,https://guide.michelin.com/en/sao-paulo-region...,https://guide.michelin.com/en/sao-paulo-region...,https://guide.michelin.com/en/sao-paulo-region...,https://guide.michelin.com/en/sao-paulo-region...,https://guide.michelin.com/en/sao-paulo-region...,https://guide.michelin.com/en/masovia/warsaw/r...,https://guide.michelin.com/en/lesser-poland/cr...,https://guide.michelin.com/en/lesser-poland/cr...,https://guide.michelin.com/en/masovia/warsaw/r...,https://guide.michelin.com/en/masovia/warsaw/r...
website_url,NaN,NaN,https://taian-table.cn/,https://uvby

In [10]:
# Display DataFrame information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6780 entries, 0 to 6779
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     6780 non-null   object 
 1   address                  6780 non-null   object 
 2   location                 6780 non-null   object 
 3   price                    6726 non-null   object 
 4   cuisine                  6780 non-null   object 
 5   longitude                6780 non-null   float64
 6   latitude                 6780 non-null   float64
 7   phone_number             6649 non-null   float64
 8   url                      6780 non-null   object 
 9   website_url              5542 non-null   object 
 10  award                    6780 non-null   object 
 11  facilities_and_services  6732 non-null   object 
 12  data                     6780 non-null   object 
dtypes: float64(3), object(10)
memory usage: 688.7+ KB


<div class="alert alert-info">

# Observations
* Dropping all nulls would remove 1336 documents (~135 documents have >1 Null)  

                          price:   54 Nulls
                   phone_number:  131 Nulls
                    website_url: 1238 Nulls
        facilities_and_services:   48 Nulls

* facilities_and_services nunique is 1571 

        * there are only 120 documents that share values with 10 or more other documents
        * there are 540 documents that share values with 2-9 other documents
        * there are 910 documents that are entirely unique

* There are only 16 locations that have more than 50 documents (awardees) and only half of those contain more than 100 documents  

        * Tokyo, Japan                    422
        * Kyoto, Japan                    196
        * Osaka, Japan                    193
        * New York, USA                   177
        * Paris, France                   164
        * Hong Kong                       135
        * Singapore                       119
        * London, United Kingdom          104
        * Bangkok, Thailand               103

* There are 863 unique cuisine values; 26 cuisine types that have more than 50 documents and only 9 that have more than 100 documents; 445 unique cuisine type combinations have only one document.  
        
        * Modern Cuisine              994
        * Creative                    416
        * Japanese                    287
        * Traditional Cuisine         218
        * French                      184
        * Street Food                 157
        * Italian                     130
        * Contemporary                114
        * Cantonese                   112

* There are four different award types  

        * Bib Gourmand        3411
        * 1 MICHELIN Star     2744
        * 2 MICHELIN Stars     483
        * 3 MICHELIN Stars     142

# Clean
* 

# Feature Engineer
* Create buckets and encode locations and cuisine
* Encode award 
* Use facilities_and_services as a feature

In [20]:
# show DataFrame without Nulls
df.shape, df.dropna().shape

((6780, 13), (5444, 13))

In [24]:
# show records where facilities_and_services is null
df[df.facilities_and_services.isnull()]

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,data
1994,Hill Street Tai Hwa Pork Noodle,"Blk 466, #01-12, Crawford Lane, 190466, Singapore",Singapore,$,Street Food,103.862400,1.305200,6.592724e+09,https://guide.michelin.com/en/singapore-region...,http://www.taihwa.com.sg,1 MICHELIN Star,NaN,Noodles are cooked-to-order with nicely layere...
4692,A Flower Blossom on the Rice,"3-6 Insadong 16-gil, Jongno-gu, Seoul, South K...","Seoul, South Korea",₩,Korean,126.984040,37.575350,8.227320e+09,https://guide.michelin.com/en/seoul-capital-ar...,https://www.goodbab.co.kr/,Bib Gourmand,NaN,Located in a quiet alley away from the tourist...
4698,Hadongkwan,"12 Myeongdong 9-gil, Jung-gu, Seoul, South Korea","Seoul, South Korea",₩,Gomtang,126.985010,37.564390,8.227766e+09,https://guide.michelin.com/en/seoul-capital-ar...,http://www.hadongkwan.com/,Bib Gourmand,NaN,"This beloved family-owned culinary landmark, l..."
4706,Myeongdong Kyoja,"29 Myeongdong 10-gil, Jung-gu, Seoul, South Korea","Seoul, South Korea",₩,Kalguksu,126.984505,37.563470,8.227765e+09,https://guide.michelin.com/en/seoul-capital-ar...,http://www.mdkj.co.kr/en/,Bib Gourmand,NaN,Myeongdong Kyoja is a family-owned place in op...
4707,Mandujip,"338 Apgujeong-ro, Gangnam-gu, Seoul, South Korea","Seoul, South Korea",₩,Mandu,127.038980,37.527850,8.225444e+09,https://guide.michelin.com/en/seoul-capital-ar...,NaN,Bib Gourmand,NaN,Located in a trendsetting part of the city dom...
4710,Yangyang Memil Makguksu,"10 Donggwang-ro 15-gil, Seocho-gu, Seoul, Sout...","Seoul, South Korea",₩,Memil-guksu,126.988260,37.492470,8.223482e+10,https://guide.michelin.com/en/seoul-capital-ar...,NaN,Bib Gourmand,NaN,The not-so-secret secret to the longevity and ...
4723,Hapjeongok,"2F, 21 Yanghwa-ro 1-gil, Mapo-gu, Seoul, South...","Seoul, South Korea",₩,Gomtang,126.910370,37.549440,8.223225e+09,https://guide.michelin.com/en/seoul-capital-ar...,NaN,Bib Gourmand,NaN,Hapjeongok is a restaurant born of nostalgia. ...
4726,Manjok Ohyang Jokbal,"134-7 Seosomun-ro, Jung-gu, Seoul, South Korea","Seoul, South Korea",₩,Jokbal,126.975980,37.563240,8.227535e+09,https://guide.michelin.com/en/seoul-capital-ar...,https://www.manjok.net/,Bib Gourmand,NaN,"At Manjok Oyhang Jokbal, the beloved Korean br..."
4727,Bongsanok,"5-6 Banpo-daero 8-gil, Seocho-gu, Seoul, South...","Seoul, South Korea",₩,Mandu,127.012450,37.483300,8.225252e+09,https://guide.michelin.com/en/seoul-capital-ar...,NaN,Bib Gourmand,NaN,"Mandutguk, or Korean dumpling soup, is a dish ..."
4730,Jin Jin,"123 Jandari-ro, Mapo-gu, Seoul, South Korea","Seoul, South Korea",₩₩,Chinese,126.913240,37.557770,8.270504e+11,https://guide.michelin.com/en/seoul-capital-ar...,NaN,Bib Gourmand,NaN,"For over 40 years, Chef Wang Yuk-sung has been..."


In [27]:
# Display count of unique entries
df.facilities_and_services.nunique()

1571

In [72]:
# Show aggregated value counts
df.facilities_and_services.value_counts().head(662)

Air conditioning,American Express credit card,Counter dining,Credit card / Debit card accepted,Diners Club credit card,JCB,Mastercard credit card,Visa credit card                                                                 308
Cash only                                                                                                                                                                                                                          165
Air conditioning,Credit card / Debit card accepted,Mastercard credit card,Visa credit card                                                                                                                                         139
Air conditioning,American Express credit card,Credit card / Debit card accepted,Mastercard credit card,Visa credit card                                                                                                            131
Credit card / Debit card accepted,Mastercard credit card,Terrace,Visa credit

In [76]:
df.award.value_counts()

Bib Gourmand        3411
1 MICHELIN Star     2744
2 MICHELIN Stars     483
3 MICHELIN Stars     142
Name: award, dtype: int64

In [84]:
# Unique of locations
df.location.nunique()

2656

In [85]:
# location counts > 50
df.location.value_counts().head(17)

Tokyo, Japan                 422
Kyoto, Japan                 196
Osaka, Japan                 193
New York, USA                177
Paris, France                164
Hong Kong                    135
Singapore                    119
London, United Kingdom       104
Bangkok, Thailand            103
Seoul, South Korea            92
Taipei, Taipei & Taichung     88
Shanghai, China Mainland      74
Chicago, USA                  73
Washington, USA               60
Guangzhou, China Mainland     57
Beijing, China Mainland       55
San Francisco, USA            46
Name: location, dtype: int64

In [118]:
df.cuisine.nunique()

863

In [126]:
# cuisine counts > 50
df.cuisine.value_counts().tail(445)

Seafood, Traditional Cuisine              2
Doganitang                                1
Central Asian                             1
Regional Cuisine, Italian Contemporary    1
Fusion, Asian                             1
                                         ..
Californian, Modern Cuisine               1
Modern British, Creative                  1
Modern Cuisine, Scottish                  1
Modern Cuisine, Irish                     1
Polish, Traditional Cuisine               1
Name: cuisine, Length: 445, dtype: int64

In [127]:
# basic cleaning function:
# add_stopwords = ['r', 'u', '2', 'ltgt']

def clean(text, stem_or_lem=None, add_stopwords=[]):
    """
    Description:
    Simplified text cleaning function
    
    Required Imports:
    import re
    import nltk
    import unicodedata
    import pandas as pd
    from nltk.corpus import stopwords

    Arguments:
             text = The text you want to clean
      stem_or_lem = Default is None; stem will perform stemming on your text; lem will lemmatize it.
    add_stopwords = [] This is an empty list by default; just add words that you want to include as stopwords.
    
    Returns:
    returns list of cleaned words.
    """
    # 1. lowercase everything
    text = text.lower()
    # 2. Remove accented and ASCII characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # 3. Remove special characters
    words = re.sub(r"[^a-z0-9\s]", '', text).split()
    # 4. Tokenize
    tokenize = nltk.tokenize.ToktokTokenizer()
    tokenize.tokenize(text, return_str=False)
    # 5. Stemming or Lemmatizing
    if stem_or_lem == "stem":
        ps = nltk.porter.PorterStemmer()
        text = [ps.stem(word) for word in text.split()]
        ' '.join(text)
        print('Stemming Performed')
    elif stem_or_lem == "lem":
        wnl = nltk.stem.WordNetLemmatizer()
        text = [wnl.lemmatize(word) for word in text.split()]
        ' '.join(text)
        print('Lemmatizing Performed')
    else:
        print('No Stemming or Lemmatizing Performed')
    # 6. Remove StopWords
    stopwords = nltk.corpus.stopwords.words('english') + add_stopwords
    
    return [word for word in words if word not in stopwords]

In [129]:
# Assign all, 1_star, 2_star, 3_star and bib_gourmand lists by passing the clean function with a join
all_reviews = clean(' '.join(df['data']),'lem')
one_star_reviews = clean(' '.join(df[df.award == '1 MICHELIN Star']['data']),'lem')
two_star_reviews = clean(' '.join(df[df.award == '2 MICHELIN Stars']['data']),'lem')
three_star_reviews = clean(' '.join(df[df.award == '3 MICHELIN Stars']['data']),'lem')
bib_gourmand_reviews = clean(' '.join(df[df.award == 'Bib Gourmand']['data']),'lem')


Lemmatizing Performed
Lemmatizing Performed
Lemmatizing Performed
Lemmatizing Performed
Lemmatizing Performed


In [130]:
# Assign all, 1_star, 2_star, 3_star and bib_gourmand lists by passing the clean function with a join
all_facilities = clean(' '.join(df['facilities_and_services']),'lem')
one_star_facilities = clean(' '.join(df[df.award == '1 MICHELIN Star']['facilities_and_services']),'lem')
two_star_facilities = clean(' '.join(df[df.award == '2 MICHELIN Stars']['facilities_and_services']),'lem')
three_star_facilities = clean(' '.join(df[df.award == '3 MICHELIN Stars']['facilities_and_services']),'lem')
bib_gourmand_facilities = clean(' '.join(df[df.award == 'Bib Gourmand']['facilities_and_services']),'lem')


TypeError: sequence item 1994: expected str instance, float found